<a href="https://colab.research.google.com/github/Anacatlisboa23/Application-of-Machine-Learning-Methodologies-in-the-Search-for-New-Peptides-with-Antimicrobial-Pote/blob/main/Criation_MIC_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Creation of the dataset for predicting MIC values taking into account the multiresistance and gram of organisms:

In [ ]:
# Importing the necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import re
import csv
from sklearn.preprocessing import StandardScaler

In [ ]:
pip install pandas openpyxl


In [ ]:
import pandas as pd


excel_file_path = '/content/Anti-Gram-_amps.xlsx'
csv_file_path = '/content/Anti-Gram-_amps.csv'
df = pd.read_excel(excel_file_path, engine='openpyxl')

df.to_csv(csv_file_path, index=False)

print(f'Ficheiro convertido e salvo em: {csv_file_path}')


Ficheiro convertido e salvo em: /content/Anti-Gram-_amps.csv


Merge datasets:

In [ ]:
neg=pd.read_csv("/content/Anti-Gram-_amps.csv")

In [ ]:
pos=pd.read_csv("/content/Anti-Gram-positiveamps.csv")

In [ ]:
merged= pd.concat([neg, pos], ignore_index=True)

In [ ]:
merged

,DRAMP_ID,Sequence,Sequence_Length,Name,Swiss_Prot_Entry,Family,Gene,Source,Activity,Protein_existence,...,N-terminal_Modification,C-terminal_Modification,Other_Modifications,Stereochemistry,Cytotoxicity,Binding_Traget,Pubmed_ID,Reference,Author,Title
0,DRAMP00089,TTKNYGNGVCNSVNWCQCGNVWASCNLATGCAAWLCKLA,39,Bacteriocin E50-52 (Preclinical),P85148,Belongs to the class IIa bacteriocin,Not found,Enterococcus faecium (Streptococcus faecium) (...,"Antimicrobial, Antibacterial, Anti-Gram+, Anti...",Protein level,...,Free,Free,NaN,L,No cytotoxicity information found,Not found,18293921,J Agric Food Chem. 2008 Mar 26;56(6):1942-1948.,"Svetoch EA, Eruslanov BV, Perelygin VV, Mitsev...",Diverse antimicrobial killing by Enterococcus ...
1,DRAMP00107,TNYGNGVGVPDAIMAGIIKLIFIFNIRQGYNFGKKAT,37,Bacteriocin L-1077,No entry found,Belongs to the class IIa bacteriocin,Not found,Lactobacillus salivarius L-1077 (NRRL B-50053)...,"Antimicrobial, Antibacterial, Anti-Gram+, Anti...",Not found,...,Free,Free,NaN,L,No cytotoxicity information found,Not found,21378051,Appl Environ Microbiol. 2011 Apr;77(8):2749-27...,"Svetoch EA, Eruslanov BV, Levchuk VP, Perelygi...",Isolation of Lactobacillus salivarius 1077 (NR...
2,DRAMP00136,NRWYCNSAAGGVGGAAVCGLAGYVGEAKENIAGEVRKGWGMAGGFT...,62,Enterocin E-760 (Bacteriocin),P85147,Belongs to the class IIb bacteriocin,Not found,Enterococcus sp. (Gram-positive bacteria),"Antimicrobial, Antibacterial, Anti-Gram+, Anti...",Protein level,...,Free,Free,NaN,L,No cytotoxicity information found,Not found,18086839,Antimicrob Agents Chemother. 2008 Mar;52(3):10...,"Line JE, Svetoch EA, Eruslanov BV, Perelygin V...",Isolation and purification of enterocin E-760 ...
3,DRAMP00171,LIDHLGAPRWAVDTILGAIAVGNLASWVLALVPGPGWAVKAGLATA...,61,Lactocyclicin Q (Bacteriocin),B9ZZY0,Belongs to the class IIc bacteriocin,lycQ,Lactococcus sp. strain QU 12 (Gram-positive ba...,"Antimicrobial, Antibacterial, Anti-Gram+, Anti...",Predicted,...,Cyclization (N termini to C termini),Cyclization (C termini to N termini),The whole peptide has a cyclic structure in wh...,L,No cytotoxicity information found,Not found,19139222##21948835,Appl Environ Microbiol. 2009 Mar;75(6):1552-15...,"Sawa N, Zendo T, Kiyofuji J, Fujita K, Himeno ...",Identification and characterization of lactocy...
4,DRAMP00191,GGAGHVPEYFVGIGTPISFYG,21,Microcin J25 (MccJ25; Bacteriocin),Q9X2V7,Belongs to the class I microcin,mcjA,Escherichia coli AY25 (Gram-negative bacteria),"Antimicrobial, Antibacterial, Anti-Gram-",Protein level,...,Cyclization of a N-terminal between Gly and Glu,Free,NaN,L,No cytotoxicity information found,Not found,1429464##10092860##14531690##14531661,J Bacteriol. 1992 Nov;174(22):7428-7435.##Eur ...,"Salom³n RA, Far­as RN.##Blond A, P©duzzi J, Go...","Microcin 25, a novel antimicrobial peptide pro..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5290,DRAMP29146,GLMGTLISKQMKK,13,Pxt‐11,No entry found,Not found,Not found,Xenopus tropicalis,"Antimicrobial, Antibacterial,Anti-Gram+, Anti-...",Not found,...,Free,Amidation,The 'G' at position 1 is pyroglutamic acid,L,[Ref.25312021]No cytotoxicity information found.,Not found,25312021,FEBS J. 2015 Jan;282(1):102-13.,"Shigeri Y, Yasuda A, Hagihara Y, Nishi K, Wata...",Identification of novel peptides from amphibia...
5291,DRAMP29147,NLLGSLLKTGLKVGSNLL,18,Pxt‐12(CPF-St7),P84386,Not found,NaN,Xenopus tropicalis,"Antimicrobial, Antibacterial,Anti-Gram+, Anti-...",Protein level,...,Free,Amidation,NaN,L,[Ref.23935531]Cytotoxicity against T-lymphocyt...,liposomes,26802742##23935531,J Biochem. 2016 Jun;159(6):619-29.##PLoS Genet...,"Shigeri Y, Horie M, Yoshida T, Hagihara Y, Imu...",Physicochemical and biological characterizatio...
5292,DRAMP29148,KLLNLLPGLLAGIF,14,Reverse Pxt‐5,No entry found,Not found,Not found,Synthetic construct,"Antimicrobial, Antibacterial,Anti-Gram+, Anti-...",Not found,...,Free,Amidation,NaN,L,[Ref.26802742]No cytotoxicity information found.,liposomes,26802742,J Bio

In [ ]:
merged.columns

Index(['DRAMP_ID', 'Sequence', 'Sequence_Length', 'Name', 'Swiss_Prot_Entry',
       'Family', 'Gene', 'Source', 'Activity', 'Protein_existence',
       'Structure', 'Structure_Description', 'PDB_ID', 'Comments',
       'Target_Organism', 'Hemolytic_activity', 'Linear/Cyclic/Branched',
       'N-terminal_Modification', 'C-terminal_Modification',
       'Other_Modifications', 'Stereochemistry', 'Cytotoxicity',
       'Binding_Traget', 'Pubmed_ID', 'Reference', 'Author', 'Title'],
      dtype='object')

Selecionar as colunas que queremos:

In [ ]:
# Select Columns
columns_to_keep = ['Sequence', 'Name', 'Swiss_Prot_Entry', 'Target_Organism']
df_filtered = merged[columns_to_keep]



In [ ]:
df_filtered

,Sequence,Name,Swiss_Prot_Entry,Target_Organism
0,TTKNYGNGVCNSVNWCQCGNVWASCNLATGCAAWLCKLA,Bacteriocin E50-52 (Preclinical),P85148,Gram-negative bacteria: Campylobacter jejuni (...
1,TNYGNGVGVPDAIMAGIIKLIFIFNIRQGYNFGKKAT,Bacteriocin L-1077,No entry found,Gram-negative bacteria: Salmonella Enteritidis...
2,NRWYCNSAAGGVGGAAVCGLAGYVGEAKENIAGEVRKGWGMAGGFT...,Enterocin E-760 (Bacteriocin),P85147,[Ref.18086839]Gram-positive bacteria: Listeria...
3,LIDHLGAPRWAVDTILGAIAVGNLASWVLALVPGPGWAVKAGLATA...,Lactocyclicin Q (Bacteriocin),B9ZZY0,Gram-positive bacteria: Lactococcus sp.strain ...
4,GGAGHVPEYFVGIGTPISFYG,Microcin J25 (MccJ25; Bacteriocin),Q9X2V7,Gram-negative bacteria: Escherichia coli BM21 ...
...,...,...,...,...
5290,GLMGTLISKQMKK,Pxt‐11,No entry found,[Ref.25312021]Gram-negative bacteria:Escherich...
5291,NLLGSLLKTGLKVGSNLL,Pxt‐12(CPF-St7),P84386,[Ref.26802742]Gram-negative bacteria:Escherich...
5292,KLLNLLPGLLAGIF,Reverse Pxt‐5,No entry found,[Ref.26802742]Gram-negative bacteria:Escherich...
5293,ALLKLAPRLLAGIF,Reverse Pxt-2,No entry found,[Ref.26802742]Gram-negative bacteria:Escherich...


In [ ]:
df_filtered.Target_Organism[34]

'[Ref.1744108]Gram-negative bacterium: 58% inhibition Neisseria meningitidis 118V C (MIC=20 µg/ml), 98% inhibition Neisseria meningitidis 15240 IAI (MIC=20 µg/ml), 98% inhibition Neisseria meningitidis 126E IC1 (MIC=20 µg/ml), 97% inhibition Neisseria gonorrhoeoe F62 (MIC=20 µg/ml), 99% inhibition Neisseria Iactamica 15323 (MIC=20 µg/ml), 100% inhibition Neisseria Iactamica 15215A (MIC=20 µg/ml), 95% inhibition Neisseria cinerea 15461 (MIC=20 µg/ml).'

Dividir organismos por colunas e organizar:

In [ ]:
# Separate Lines By Organisms:

df_subset = df_filtered[['Sequence', 'Name','Target_Organism']].copy()

# Separate the organisms' information into individual lines
new_rows = []
for index, row in df_subset.iterrows():
    organisms = row['Target_Organism'].replace(';', ',').split(',')
    #organisms = row['Target_Organism'].split(',')
    for organism in organisms:
        new_row = row.copy()
        new_row['Target_Organism'] = organism.strip()
        new_rows.append(new_row)


df_filtered = pd.DataFrame(new_rows)
df_filtered


,Sequence,Name,Target_Organism
0,TTKNYGNGVCNSVNWCQCGNVWASCNLATGCAAWLCKLA,Bacteriocin E50-52 (Preclinical),Gram-negative bacteria: Campylobacter jejuni (...
0,TTKNYGNGVCNSVNWCQCGNVWASCNLATGCAAWLCKLA,Bacteriocin E50-52 (Preclinical),Yersinia enterocolitica
0,TTKNYGNGVCNSVNWCQCGNVWASCNLATGCAAWLCKLA,Bacteriocin E50-52 (Preclinical),Yersinia pseudotuberculosis
0,TTKNYGNGVCNSVNWCQCGNVWASCNLATGCAAWLCKLA,Bacteriocin E50-52 (Preclinical),Escherichia coli O157:H7 (MIC<1.6 µg/ml)
0,TTKNYGNGVCNSVNWCQCGNVWASCNLATGCAAWLCKLA,Bacteriocin E50-52 (Preclinical),##Gram-positive bacteria: Staphylococcus aureu...
...,...,...,...
5294,LLNSGVKLGTKLLSGLLN,Reverse Pxt-12,[Ref.26802742]Gram-negative bacteria:Escherich...
5294,LLNSGVKLGTKLLSGLLN,Reverse Pxt-12,Pseudomonas aeruginosa NBRC 12582(ND)
5294,LLNSGVKLGTKLLSGLLN,Reverse Pxt-12,##Gram-positive bacteria:Staphylococcus aureus...
5294,LLNSGVKLGTKLLSGLLN,Reverse Pxt-12,Micrococcus luteus NBRC 12708(ND)


In [ ]:
#
# Step 1: Split organisms based on '##' delimiter
df_filtered['Target_Organism'] = df_filtered['Target_Organism'].str.split('##')
#
# # Step 2 and 3: Duplicate rows for each organism and update the 'Target_Organism' column
df_filtrado = df_filtered.explode('Target_Organism')
#
# # Step 4: Optionally, strip leading and trailing whitespaces
#df_filtrado_neg['Target_Organism'] = df_filtrado_neg['Target_Organism'].str.strip()
df_filtrado


,Sequence,Name,Target_Organism
0,TTKNYGNGVCNSVNWCQCGNVWASCNLATGCAAWLCKLA,Bacteriocin E50-52 (Preclinical),Gram-negative bacteria: Campylobacter jejuni (...
0,TTKNYGNGVCNSVNWCQCGNVWASCNLATGCAAWLCKLA,Bacteriocin E50-52 (Preclinical),Yersinia enterocolitica
0,TTKNYGNGVCNSVNWCQCGNVWASCNLATGCAAWLCKLA,Bacteriocin E50-52 (Preclinical),Yersinia pseudotuberculosis
0,TTKNYGNGVCNSVNWCQCGNVWASCNLATGCAAWLCKLA,Bacteriocin E50-52 (Preclinical),Escherichia coli O157:H7 (MIC<1.6 µg/ml)
0,TTKNYGNGVCNSVNWCQCGNVWASCNLATGCAAWLCKLA,Bacteriocin E50-52 (Preclinical),
...,...,...,...
5294,LLNSGVKLGTKLLSGLLN,Reverse Pxt-12,
5294,LLNSGVKLGTKLLSGLLN,Reverse Pxt-12,Gram-positive bacteria:Staphylococcus aureus N...
5294,LLNSGVKLGTKLLSGLLN,Reverse Pxt-12,Micrococcus luteus NBRC 12708(ND)
5294,LLNSGVKLGTKLLSGLLN,Reverse Pxt-12,


In [ ]:
#Remove the text before the colon
def remove_text_before_colon(text):
    return re.sub(r'^[^:]*:\s*', '', text)


df_filtrado['Target_Organism'] = df_filtrado['Target_Organism'].apply(remove_text_before_colon)
df_filtrado

,Sequence,Name,Target_Organism
0,TTKNYGNGVCNSVNWCQCGNVWASCNLATGCAAWLCKLA,Bacteriocin E50-52 (Preclinical),Campylobacter jejuni (MIC=0.025-6.4 µg/ml)
0,TTKNYGNGVCNSVNWCQCGNVWASCNLATGCAAWLCKLA,Bacteriocin E50-52 (Preclinical),Yersinia enterocolitica
0,TTKNYGNGVCNSVNWCQCGNVWASCNLATGCAAWLCKLA,Bacteriocin E50-52 (Preclinical),Yersinia pseudotuberculosis
0,TTKNYGNGVCNSVNWCQCGNVWASCNLATGCAAWLCKLA,Bacteriocin E50-52 (Preclinical),H7 (MIC<1.6 µg/ml)
0,TTKNYGNGVCNSVNWCQCGNVWASCNLATGCAAWLCKLA,Bacteriocin E50-52 (Preclinical),
...,...,...,...
5294,LLNSGVKLGTKLLSGLLN,Reverse Pxt-12,
5294,LLNSGVKLGTKLLSGLLN,Reverse Pxt-12,Staphylococcus aureus NBRC 12732(ND)
5294,LLNSGVKLGTKLLSGLLN,Reverse Pxt-12,Micrococcus luteus NBRC 12708(ND)
5294,LLNSGVKLGTKLLSGLLN,Reverse Pxt-12,


Save the MIC values:

In [ ]:
# Function to extract MIC values from the ‘Target_Organism’ column
def extract_mic(entry):
    # Find all MIC values in the string
    mic_matches = re.findall(r'\(MIC[<=]?([^\)]+)\)', entry)
    return mic_matches

# Add a column with the extracted MIC values
df_filtrado['MIC_Values'] = df_filtrado['Target_Organism'].apply(lambda x: extract_mic(x))


df_mic_filtered = df_filtrado[df_filtrado['MIC_Values'].apply(lambda x: len(x) > 0)]

In [ ]:
# Count the rows that have values in the ‘MIC’ column
mic_count = df_filtrado['MIC_Values'].notna().sum()
mic_count

39190

Save the ones we need:

In [ ]:
eskape_organisms = [
    'Enterococcus faecium',
    'Staphylococcus aureus',
    'Klebsiella pneumoniae',
    'Acinetobacter baumannii',
    'Pseudomonas aeruginosa',
    'Enterobacter'
]

# Create a regular expression to match any ESKAPE organism
eskape_regex = '|'.join(eskape_organisms)

df_eskape_filtered = df_mic_filtered[df_mic_filtered['Target_Organism'].str.contains(eskape_regex, case=False, na=False)]

In [ ]:
df_eskape_filtered

,Sequence,Name,Target_Organism,MIC_Values
0,TTKNYGNGVCNSVNWCQCGNVWASCNLATGCAAWLCKLA,Bacteriocin E50-52 (Preclinical),Staphylococcus aureus (MIC=0.2-0.8 µg/ml),[0.2-0.8 µg/ml]
1,TNYGNGVGVPDAIMAGIIKLIFIFNIRQGYNFGKKAT,Bacteriocin L-1077,Klebsiella pneumoniae (MIC=0.76 ug/ml),[0.76 ug/ml]
1,TNYGNGVGVPDAIMAGIIKLIFIFNIRQGYNFGKKAT,Bacteriocin L-1077,Pseudomonas aeruginosa 508 (MIC=0.38 ug/ml),[0.38 ug/ml]
1,TNYGNGVGVPDAIMAGIIKLIFIFNIRQGYNFGKKAT,Bacteriocin L-1077,Staphylococcus aureus (MIC=0.76 ug/ml),[0.76 ug/ml]
2,NRWYCNSAAGGVGGAAVCGLAGYVGEAKENIAGEVRKGWGMAGGFT...,Enterocin E-760 (Bacteriocin),Klebsiella pneumoniae (MIC=3.2 μg/ml),[3.2 μg/ml]
...,...,...,...,...
5291,NLLGSLLKTGLKVGSNLL,Pxt‐12(CPF-St7),Staphylococcus aureus ATCC 25923(MIC=32 µM),[32 µM]
5292,KLLNLLPGLLAGIF,Reverse Pxt‐5,Pseudomonas aeruginosa NBRC 12582(MIC<50 µM),[50 µM]
5292,KLLNLLPGLLAGIF,Reverse Pxt‐5,Staphylococcus aureus NBRC 12732(MIC<12.5 µM),[12.5 µM]
5293,ALLKLAPRLLAGIF,Reverse Pxt-2,Pseudomonas aeruginosa NBRC 12582(MIC<50 µM),[50 µM]


In [ ]:
# Filter the lines that have non-null MIC values
df_mic_not_null = df_eskape_filtered[df_eskape_filtered['MIC_Values'].notna()]

In [ ]:
df_mic_not_null

,Sequence,Name,Target_Organism,MIC_Values
0,TTKNYGNGVCNSVNWCQCGNVWASCNLATGCAAWLCKLA,Bacteriocin E50-52 (Preclinical),Staphylococcus aureus (MIC=0.2-0.8 µg/ml),[0.2-0.8 µg/ml]
1,TNYGNGVGVPDAIMAGIIKLIFIFNIRQGYNFGKKAT,Bacteriocin L-1077,Klebsiella pneumoniae (MIC=0.76 ug/ml),[0.76 ug/ml]
1,TNYGNGVGVPDAIMAGIIKLIFIFNIRQGYNFGKKAT,Bacteriocin L-1077,Pseudomonas aeruginosa 508 (MIC=0.38 ug/ml),[0.38 ug/ml]
1,TNYGNGVGVPDAIMAGIIKLIFIFNIRQGYNFGKKAT,Bacteriocin L-1077,Staphylococcus aureus (MIC=0.76 ug/ml),[0.76 ug/ml]
2,NRWYCNSAAGGVGGAAVCGLAGYVGEAKENIAGEVRKGWGMAGGFT...,Enterocin E-760 (Bacteriocin),Klebsiella pneumoniae (MIC=3.2 μg/ml),[3.2 μg/ml]
...,...,...,...,...
5291,NLLGSLLKTGLKVGSNLL,Pxt‐12(CPF-St7),Staphylococcus aureus ATCC 25923(MIC=32 µM),[32 µM]
5292,KLLNLLPGLLAGIF,Reverse Pxt‐5,Pseudomonas aeruginosa NBRC 12582(MIC<50 µM),[50 µM]
5292,KLLNLLPGLLAGIF,Reverse Pxt‐5,Staphylococcus aureus NBRC 12732(MIC<12.5 µM),[12.5 µM]
5293,ALLKLAPRLLAGIF,Reverse Pxt-2,Pseudomonas aeruginosa NBRC 12582(MIC<50 µM),[50 µM]


In [ ]:
df_mic_not_null.Target_Organism

,Target_Organism
0,Staphylococcus aureus (MIC=0.2-0.8 µg/ml)
1,Klebsiella pneumoniae (MIC=0.76 ug/ml)
1,Pseudomonas aeruginosa 508 (MIC=0.38 ug/ml)
1,Staphylococcus aureus (MIC=0.76 ug/ml)
2,Klebsiella pneumoniae (MIC=3.2 μg/ml)
...,...
5291,Staphylococcus aureus ATCC 25923(MIC=32 µM)
5292,Pseudomonas aeruginosa NBRC 12582(MIC<50 µM)
5292,Staphylococcus aureus NBRC 12732(MIC<12.5 µM)
5293,Pseudomonas aeruginosa NBRC 12582(MIC<50 µM)


In [ ]:
# Function to remove text inside brackets
def remove_parentheses_content(text):
    return re.sub(r'\(MIC[^\)]+\)', '', text).strip()

df_mic_not_null['Target_Organism'] = df_mic_not_null['Target_Organism'].apply(remove_parentheses_content)


In [ ]:
df_mic_not_null

,Sequence,Name,Target_Organism,MIC_Values
0,TTKNYGNGVCNSVNWCQCGNVWASCNLATGCAAWLCKLA,Bacteriocin E50-52 (Preclinical),Staphylococcus aureus,[0.2-0.8 µg/ml]
1,TNYGNGVGVPDAIMAGIIKLIFIFNIRQGYNFGKKAT,Bacteriocin L-1077,Klebsiella pneumoniae,[0.76 ug/ml]
1,TNYGNGVGVPDAIMAGIIKLIFIFNIRQGYNFGKKAT,Bacteriocin L-1077,Pseudomonas aeruginosa 508,[0.38 ug/ml]
1,TNYGNGVGVPDAIMAGIIKLIFIFNIRQGYNFGKKAT,Bacteriocin L-1077,Staphylococcus aureus,[0.76 ug/ml]
2,NRWYCNSAAGGVGGAAVCGLAGYVGEAKENIAGEVRKGWGMAGGFT...,Enterocin E-760 (Bacteriocin),Klebsiella pneumoniae,[3.2 μg/ml]
...,...,...,...,...
5291,NLLGSLLKTGLKVGSNLL,Pxt‐12(CPF-St7),Staphylococcus aureus ATCC 25923,[32 µM]
5292,KLLNLLPGLLAGIF,Reverse Pxt‐5,Pseudomonas aeruginosa NBRC 12582,[50 µM]
5292,KLLNLLPGLLAGIF,Reverse Pxt‐5,Staphylococcus aureus NBRC 12732,[12.5 µM]
5293,ALLKLAPRLLAGIF,Reverse Pxt-2,Pseudomonas aeruginosa NBRC 12582,[50 µM]


In [ ]:
df_mic_not_null.to_csv("/content/anti-ESKAPE_MIC_Final.csv", index=False)

In [ ]:
 # Define the gram type for each ESKAPE organism
gram_types = {
    'Enterococcus faecium': 'Gram-positivo',
    'Staphylococcus aureus': 'Gram-positivo',
    'Klebsiella pneumoniae': 'Gram-negativo',
    'Acinetobacter baumannii': 'Gram-negativo',
    'Pseudomonas aeruginosa': 'Gram-negativo',
    'Enterobacter': 'Gram-negativo'
}

# Function to identify the type of gram
def identify_gram(organism_name):
    for key in gram_types:
        if key in organism_name:
            return gram_types[key]
    return 'Desconhecido'


df_mic_not_null['Gram'] = df_mic_not_null['Target_Organism'].apply(identify_gram)

In [ ]:
df_mic_not_null

,Sequence,Name,Target_Organism,MIC_Values,Gram
0,TTKNYGNGVCNSVNWCQCGNVWASCNLATGCAAWLCKLA,Bacteriocin E50-52 (Preclinical),Staphylococcus aureus,[0.2-0.8 µg/ml],Gram-positivo
1,TNYGNGVGVPDAIMAGIIKLIFIFNIRQGYNFGKKAT,Bacteriocin L-1077,Klebsiella pneumoniae,[0.76 ug/ml],Gram-negativo
1,TNYGNGVGVPDAIMAGIIKLIFIFNIRQGYNFGKKAT,Bacteriocin L-1077,Pseudomonas aeruginosa 508,[0.38 ug/ml],Gram-negativo
1,TNYGNGVGVPDAIMAGIIKLIFIFNIRQGYNFGKKAT,Bacteriocin L-1077,Staphylococcus aureus,[0.76 ug/ml],Gram-positivo
2,NRWYCNSAAGGVGGAAVCGLAGYVGEAKENIAGEVRKGWGMAGGFT...,Enterocin E-760 (Bacteriocin),Klebsiella pneumoniae,[3.2 μg/ml],Gram-negativo
...,...,...,...,...,...
5291,NLLGSLLKTGLKVGSNLL,Pxt‐12(CPF-St7),Staphylococcus aureus ATCC 25923,[32 µM],Gram-positivo
5292,KLLNLLPGLLAGIF,Reverse Pxt‐5,Pseudomonas aeruginosa NBRC 12582,[50 µM],Gram-negativo
5292,KLLNLLPGLLAGIF,Reverse Pxt‐5,Staphylococcus aureus NBRC 12732,[12.5 µM],Gram-positivo
5293,ALLKLAPRLLAGIF,Reverse Pxt-2,Pseudomonas aeruginosa NBRC 12582,[50 µM],Gram-negativo


In [ ]:
df_mic_not_null.to_csv("/content/anti_eskape_mic_final_gram.csv", index=False)

In [ ]:
unique_organisms = df_mic_not_null['Target_Organism'].unique()

In [ ]:
unique_organisms

array(['Staphylococcus aureus', 'Klebsiella pneumoniae',
       'Pseudomonas aeruginosa 508', 'Enterococcus faecium JCM5804',
       'Staphylococcus aureus subsp.aureus ATCC 12600',
       'Staphylococcus aureus ATTC 25923',
       'Staphylococcus aureus ATCC 25923', 'Pseudomonas aeruginosa PA01',
       'Pseudomonas aeruginosa ATCC 27853',
       'Pseudomonas aeruginosa 08031014 (CI) .',
       'Pseudomonas aeruginosa ATCC 27553',
       'Pseudomonas aeruginosa ATCC 39018', 'Staphylococcus aureus .',
       '(L-salt): Staphylococcus aureus .', 'Klebsiella pneumoniae .',
       'Pseudomonas aeruginosa',
       'Escherichia coli NCTC 10418  and Gram-positive bacterium: Staphylococcus aureus NCTC 10788 .',
       'Staphylococcus aureus ATCC2592', 'Staphylococcus aureus 6A',
       'Staphylococcus aureus 15A', 'Staphylococcus aureus Cowan1',
       'Pseudomonas aeruginosa ATCC 15692',
       'Staphylococcus aureus Cowan 1 .', 'Enterobacter cloacae',
       'Klebsiella pneumoniae KK3 9904'

In [ ]:
organismos_para_remover = ['Clostridium difficile', 'Escherichia coli', 'Mycobacterium tuberculosis']


df_filtrado = df_mic_not_null[~df_mic_not_null['Target_Organism'].isin(organismos_para_remover)]


df_filtrado.to_csv('seu_arquivo_filtrado.csv', index=False)


In [ ]:
df_filtrado

,Sequence,Name,Target_Organism,MIC_Values,Gram
0,TTKNYGNGVCNSVNWCQCGNVWASCNLATGCAAWLCKLA,Bacteriocin E50-52 (Preclinical),Staphylococcus aureus,[0.2-0.8 µg/ml],Gram-positivo
1,TNYGNGVGVPDAIMAGIIKLIFIFNIRQGYNFGKKAT,Bacteriocin L-1077,Klebsiella pneumoniae,[0.76 ug/ml],Gram-negativo
1,TNYGNGVGVPDAIMAGIIKLIFIFNIRQGYNFGKKAT,Bacteriocin L-1077,Pseudomonas aeruginosa 508,[0.38 ug/ml],Gram-negativo
1,TNYGNGVGVPDAIMAGIIKLIFIFNIRQGYNFGKKAT,Bacteriocin L-1077,Staphylococcus aureus,[0.76 ug/ml],Gram-positivo
2,NRWYCNSAAGGVGGAAVCGLAGYVGEAKENIAGEVRKGWGMAGGFT...,Enterocin E-760 (Bacteriocin),Klebsiella pneumoniae,[3.2 μg/ml],Gram-negativo
...,...,...,...,...,...
5291,NLLGSLLKTGLKVGSNLL,Pxt‐12(CPF-St7),Staphylococcus aureus ATCC 25923,[32 µM],Gram-positivo
5292,KLLNLLPGLLAGIF,Reverse Pxt‐5,Pseudomonas aeruginosa NBRC 12582,[50 µM],Gram-negativo
5292,KLLNLLPGLLAGIF,Reverse Pxt‐5,Staphylococcus aureus NBRC 12732,[12.5 µM],Gram-positivo
5293,ALLKLAPRLLAGIF,Reverse Pxt-2,Pseudomonas aeruginosa NBRC 12582,[50 µM],Gram-negativo


Multi-resistant:

In [ ]:
# List of patterns that indicate multi-resistance
multiresistant_patterns = [
    'MDR', 'MRSA', 'NDM-1', 'XDR', 'PDR', 'ESBL', 'KPC', 'VRE', 'CRE', 'MDR-TB', 'XDR-TB'
]

# Function to check if the name contains any of the multidrug resistance standards
def is_multiresistant(organism_name):
    return any(pattern in organism_name for pattern in multiresistant_patterns)

# Apply the function to create a new column indicating whether it is multi-resistant
df_filtrado['multiresistant'] = df_filtrado['Target_Organism'].apply(is_multiresistant)


df_filtrado.to_csv('organismos_atualizados.csv', index=False)


print("DataFrame atualizado com coluna de multirresistência:")
print(df_filtrado)

DataFrame atualizado com coluna de multirresistência:
                                               Sequence  \
0               TTKNYGNGVCNSVNWCQCGNVWASCNLATGCAAWLCKLA   
1                 TNYGNGVGVPDAIMAGIIKLIFIFNIRQGYNFGKKAT   
1                 TNYGNGVGVPDAIMAGIIKLIFIFNIRQGYNFGKKAT   
1                 TNYGNGVGVPDAIMAGIIKLIFIFNIRQGYNFGKKAT   
2     NRWYCNSAAGGVGGAAVCGLAGYVGEAKENIAGEVRKGWGMAGGFT...   
...                                                 ...   
5291                                 NLLGSLLKTGLKVGSNLL   
5292                                     KLLNLLPGLLAGIF   
5292                                     KLLNLLPGLLAGIF   
5293                                     ALLKLAPRLLAGIF   
5293                                     ALLKLAPRLLAGIF   

                                  Name                    Target_Organism  \
0     Bacteriocin E50-52 (Preclinical)              Staphylococcus aureus   
1                   Bacteriocin L-1077              Klebsiella pneumoniae   
1     

In [ ]:
df_filtrado.to_csv("/content/anti_eskape_multiresistencia_gram_mic.csv",index=False)

In [ ]:
# Contar quantos True existem na coluna multiresistant
num_multiresistant = df['multiresistant'].sum()
num_multiresistant

211